<a href="https://colab.research.google.com/github/KexinGao-Augustine/UD_IntroProject_2022/blob/main/Segment_for_concertration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Auto-generating mask

In [ ]:
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
sys.path.append("..")
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

!mkdir images
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
    
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

PyTorch version: 2.0.0+cu118
Torchvision version: 0.15.1+cu118
CUDA is available: True
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-20hwqd2l
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-20hwqd2l
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
mkdir: cannot create directory ‘images’: File exists
--2023-05-05 21:15:44--  https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.g

In [ ]:
!pip install onnxruntime
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 78.8 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import onnxruntime
from onnxruntime.quantization import QuantType
from onnxruntime.quantization.quantize import quantize_dynamic

from segment_anything.utils.onnx import SamOnnxModel
from segment_anything import sam_model_registry, SamPredictor, SamAutomaticMaskGenerator
sam = sam_model_registry["vit_h"](checkpoint="/content/drive/MyDrive/for_masks/sam_vit_h_4b8939.pth")
sam.to(device = "cuda")
mask_generator = SamAutomaticMaskGenerator(sam)

##One image

In [ ]:
image = '/content/drive/MyDrive/for_masks/demo/output_229.jpg'
img = cv2.imread(image)
masks = mask_generator.generate(img) #the generated mask

In [ ]:
#masks can be generated for images from the command line
!python /content/drive/MyDrive/for_masks/amg.py --checkpoint /content/drive/MyDrive/for_masks/sam_vit_h_4b8939.pth --model-type vit_h --input /content/drive/MyDrive/for_masks/demo/output_229.jpg --output /content/drive/MyDrive/for_masks/demo/output_229_mask.png

Loading model...
Traceback (most recent call last):
  File "/content/drive/MyDrive/for_masks/amg.py", line 238, in <module>
    main(args)
  File "/content/drive/MyDrive/for_masks/amg.py", line 198, in main
    _ = sam.to(device=args.device)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1145, in to
    return self._apply(convert)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 797, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 797, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 797, in _apply
    module._apply(fn)
  [Previous line repeated 2 more times]
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 820, in _apply
    param_applied = fn(param)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1143, in convert
    retur

In [ ]:
#Export the model with onnx format
#it's a mistake to use this command for we don't need onnx format, yet it didn't run successfully
#forget this and focus on the above one
!python /content/drive/MyDrive/for_masks/export_onnx_model.py --checkpoint /content/drive/MyDrive/for_masks/sam_vit_h_4b8939.pth --model-type vit_h --output /content/drive/MyDrive/for_masks/save/01.png

Loading model...
Exporting onnx model to /content/drive/MyDrive/for_masks/save/01.png...
============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/onnx/_internal/onnx_proto_utils.py", line 219, in _add_onnxscript_fn
    import onnx
ModuleNotFoundError: No module named 'onnx'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/content/drive/MyDrive/for_masks/export_onnx_model.py", line 176, in <module>
    run_export(
  File "/content/drive/MyDrive/for_masks/export_onnx_model.py", line 148, in run_export
    torch.onnx.export(
  File "/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py", line 506, in export
    _export(
  File "/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py",

##Bach processing

In [ ]:
folder = "/content/drive/MyDrive/for_masks/demo" #change the path
image_list = os.listdir(folder)
images = []
for image in image_list:
  images.append(os.path.join(folder, image))
images

['/content/drive/MyDrive/for_masks/demo/output_230.jpg',
 '/content/drive/MyDrive/for_masks/demo/output_231.jpg',
 '/content/drive/MyDrive/for_masks/demo/output_229.jpg']

In [ ]:
for i in images:
  img = cv2.imread(i)
  mask_path = os.path.join("/content/drive/MyDrive/for_masks/save", f"{os.path.splitext(i)[0]}_mask.onnx")
  print(i)
  print(mask_path)
  masks = mask_generator.generate(img)
  !python /content/drive/MyDrive/for_masks/export_onnx_model.py --checkpoint /content/drive/MyDrive/for_masks/sam_vit_h_4b8939.pth --model-type vit_h --output $mask_path

/content/drive/MyDrive/for_masks/demo/output_230.jpg
/content/drive/MyDrive/for_masks/demo/output_230_mask.onnx
Loading model...
Exporting onnx model to /content/drive/MyDrive/for_masks/demo/output_230_mask.onnx...
============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/onnx/_internal/onnx_proto_utils.py", line 219, in _add_onnxscript_fn
    import onnx
ModuleNotFoundError: No module named 'onnx'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/content/drive/MyDrive/for_masks/export_onnx_model.py", line 176, in <module>
    run_export(
  File "/content/drive/MyDrive/for_masks/export_onnx_model.py", line 148, in run_export
    torch.onnx.export(
  File "/usr/local/lib/python3.10/dist-package